In [1]:
import re 
import os
from datetime import datetime 
import string

import numpy as np
import pandas as pd

from praw.models import MoreComments
import praw

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import *

from wordcloud import WordCloud

from textblob import TextBlob

import warnings
warnings.filterwarnings('ignore')
np.random.seed(37)

In [2]:
#!pip install langdetect

In [2]:
df = pd.read_csv('245_1.csv')

In [3]:
data = df.copy()

In [4]:
data.head()

,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,upc
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71044 entries, 0 to 71043
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    71044 non-null  object 
 1   brand                 71044 non-null  object 
 2   categories            71044 non-null  object 
 3   dateAdded             71044 non-null  object 
 4   dateUpdated           71044 non-null  object 
 5   ean                   39065 non-null  object 
 6   keys                  71044 non-null  object 
 7   manufacturer          70903 non-null  object 
 8   manufacturerNumber    70841 non-null  object 
 9   name                  71044 non-null  object 
 10  reviews.date          70977 non-null  object 
 11  reviews.dateAdded     71044 non-null  object 
 12  reviews.dateSeen      71044 non-null  object 
 13  reviews.didPurchase   32158 non-null  object 
 14  reviews.doRecommend   60429 non-null  object 
 15  reviews.id         

In [4]:
data.drop(['brand', 'categories', 'dateAdded', 'dateUpdated', 'ean', 'keys', 'manufacturer', 'manufacturerNumber', 'name', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.numHelpful', 'reviews.sourceURLs', 'reviews.title', 'reviews.userCity', 'reviews.userProvince', 'reviews.username', 'upc'], axis=1, inplace=True)

In [7]:
data

,id,reviews.date,reviews.id,reviews.rating,reviews.text
0,AV13O1A8GV-KLJ3akUyj,2012-11-30T06:21:45.000Z,NaN,5,i love this album. it's very good. more to the...
1,AV14LG0R-jtxr-f38QfS,2017-07-09T00:00:00.000Z,100209113.0,5,Good flavor. This review was collected as part...
2,AV14LG0R-jtxr-f38QfS,2017-07-09T00:00:00.000Z,100209113.0,5,Good flavor.
3,AV16khLE-jtxr-f38VFn,2016-01-06T00:00:00.000Z,113026909.0,1,I read through the reviews on here before look...
4,AV16khLE-jtxr-f38VFn,2016-12-21T00:00:00.000Z,171267657.0,1,My husband bought this gel for us. The gel cau...
...,...,...,...,...,...
71039,AVzRGDlw-jtxr-f3yfFH,2012-03-25T00:00:00.000Z,6855681.0,5,I'm dealing with dry eyes and very sensitive t...
71040,AVzRGDlw-jtxr-f3yfFH,2015-12-07T00:00:00.000Z,71102868.0,5,Great stuff! Helps keep my contacts moist all ...
71041,AVzRGDlw-jtxr-f3yfFH,2015-07-23T00:00:00.000Z,58039428.0,5,Right price
71042,AVzvTLfivKc47QAVfPme,2016-01-19T18:28:16.000Z,NaN,5,I was so pleased when this scarf arrived for m...


In [5]:
data = data.drop(data[data["reviews.text"]== '[removed]'].index)

In [9]:
data

,id,reviews.date,reviews.id,reviews.rating,reviews.text
0,AV13O1A8GV-KLJ3akUyj,2012-11-30T06:21:45.000Z,NaN,5,i love this album. it's very good. more to the...
1,AV14LG0R-jtxr-f38QfS,2017-07-09T00:00:00.000Z,100209113.0,5,Good flavor. This review was collected as part...
2,AV14LG0R-jtxr-f38QfS,2017-07-09T00:00:00.000Z,100209113.0,5,Good flavor.
3,AV16khLE-jtxr-f38VFn,2016-01-06T00:00:00.000Z,113026909.0,1,I read through the reviews on here before look...
4,AV16khLE-jtxr-f38VFn,2016-12-21T00:00:00.000Z,171267657.0,1,My husband bought this gel for us. The gel cau...
...,...,...,...,...,...
71039,AVzRGDlw-jtxr-f3yfFH,2012-03-25T00:00:00.000Z,6855681.0,5,I'm dealing with dry eyes and very sensitive t...
71040,AVzRGDlw-jtxr-f3yfFH,2015-12-07T00:00:00.000Z,71102868.0,5,Great stuff! Helps keep my contacts moist all ...
71041,AVzRGDlw-jtxr-f3yfFH,2015-07-23T00:00:00.000Z,58039428.0,5,Right price
71042,AVzvTLfivKc47QAVfPme,2016-01-19T18:28:16.000Z,NaN,5,I was so pleased when this scarf arrived for m...


In [6]:
data=data.sort_values(by='reviews.date',ascending=False)
data

,id,reviews.date,reviews.id,reviews.rating,reviews.text
36796,AVpgKuphilAPnD_xodXm,2018-01-19T00:00:00.000Z,NaN,5,Sense we got this can opener we have had no is...
36795,AVpgKuphilAPnD_xodXm,2018-01-15T00:00:00.000Z,NaN,5,Ive had this Can opener for very long time and...
36461,AVpghlTX1cnluZ0-2rIC,2018-01-14T16:52:33.000Z,NaN,5,This product is very sturdy. My new kitten use...
3503,AVpe625_ilAPnD_xQ4SP,2018-01-10T19:56:37.000Z,NaN,2,I was disappointed when I received this swag I...
57707,AVphMOeVLJeJML43X5ut,2018-01-07T16:48:31.000Z,NaN,5,"High quality, great appearance"
...,...,...,...,...,...
36457,AVpggt6m1cnluZ0-2eYE,NaN,NaN,1,Too large to fit inside of my civic's stock mi...
38874,AVpgmXOsLJeJML43QMSJ,NaN,NaN,5,"This is not my first purchase for this one, it..."
46588,AVpgS0cdLJeJML43L490,NaN,NaN,5,great service..great bargain..perfect conditio...
47374,AVpgTzVT1cnluZ0-zrZQ,NaN,NaN,5,I bought this from Amazon when I discovered Bu...


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71044 entries, 36796 to 47425
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              71044 non-null  object 
 1   reviews.date    70977 non-null  object 
 2   reviews.id      32158 non-null  float64
 3   reviews.rating  71044 non-null  int64  
 4   reviews.text    71008 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.3+ MB


In [7]:
data.drop_duplicates(subset ="reviews.text",keep=False,inplace=True)

In [8]:
data["cleaned_comments"] = data["reviews.text"].str.replace(r"[/]*u[/][\w]*", ' ').str.strip()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59097 entries, 36796 to 47425
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                59097 non-null  object 
 1   reviews.date      59032 non-null  object 
 2   reviews.id        25540 non-null  float64
 3   reviews.rating    59097 non-null  int64  
 4   reviews.text      59097 non-null  object 
 5   cleaned_comments  59097 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 3.2+ MB


In [15]:
from langdetect import detect

def detect_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [ ]:
data = data[data["cleaned_comments"].apply(detect_english)]

In [9]:
data=pd.DataFrame(data)
data

,id,reviews.date,reviews.id,reviews.rating,reviews.text,cleaned_comments
36796,AVpgKuphilAPnD_xodXm,2018-01-19T00:00:00.000Z,NaN,5,Sense we got this can opener we have had no is...,Sense we got this can opener we have had no is...
36795,AVpgKuphilAPnD_xodXm,2018-01-15T00:00:00.000Z,NaN,5,Ive had this Can opener for very long time and...,Ive had this Can opener for very long time and...
36461,AVpghlTX1cnluZ0-2rIC,2018-01-14T16:52:33.000Z,NaN,5,This product is very sturdy. My new kitten use...,This product is very sturdy. My new kitten use...
3503,AVpe625_ilAPnD_xQ4SP,2018-01-10T19:56:37.000Z,NaN,2,I was disappointed when I received this swag I...,I was disappointed when I received this swag I...
57707,AVphMOeVLJeJML43X5ut,2018-01-07T16:48:31.000Z,NaN,5,"High quality, great appearance","High quality, great appearance"
...,...,...,...,...,...,...
36457,AVpggt6m1cnluZ0-2eYE,NaN,NaN,1,Too large to fit inside of my civic's stock mi...,Too large to fit inside of my civic's stock mi...
38874,AVpgmXOsLJeJML43QMSJ,NaN,NaN,5,"This is not my first purchase for this one, it...","This is not my first purchase for this one, it..."
46588,AVpgS0cdLJeJML43L490,NaN,NaN,5,great service..great bargain..perfect conditio...,great service..great bargain..perfect conditio...
47374,AVpgTzVT1cnluZ0-zrZQ,NaN,NaN,5,I bought this from Amazon when I discovered Bu...,I bought this from Amazon when I discovered Bu...


In [10]:
no_http = []

data["cleaned_comments"] = data["cleaned_comments"].str.replace(r'.{0,1}\s*https?://\S+(\s+|$)', ' ').str.strip()

In [11]:
data["cleaned_comments"] = data['cleaned_comments'].str.replace('[^\w\s]','')
data["cleaned_comments"] = data['cleaned_comments'].str.replace('[/\n]*','')

In [12]:
data[['reviews.text','cleaned_comments']].tail(200)

,reviews.text,cleaned_comments
32501,"Nice mix of kickboxing, cardio, and strength t...",Nice mix of kickboxing cardio and strength tra...
4534,"Easy Assembly, great features...auto height ad...",Easy Assembly great featuresauto height adjust...
4550,"Absoluty love this vaccum, well worth every pe...",Absoluty love this vaccum well worth every pen...
17378,"Since I've stopped eating wheat products, I've...",Since Ive stopped eating wheat products Ive be...
61215,Half of this product is great. The Mask movie ...,Half of this product is great The Mask movie i...
...,...,...
36457,Too large to fit inside of my civic's stock mi...,Too large to fit inside of my civics stock mir...
38874,"This is not my first purchase for this one, it...",This is not my first purchase for this one it ...
46588,great service..great bargain..perfect conditio...,great servicegreat bargainperfect conditiontha...
47374,I bought this from Amazon when I discovered Bu...,I bought this from Amazon when I discovered Bu...


In [ ]:
#data.to_csv("cleaned_redit_data.csv")

In [19]:
nltk.download('stopwords') #English words which does not add much meaning to a sentence.
nltk.download('wordnet')#large word database
nltk.download('vader_lexicon')#sentiment analysis tool
nltk.download('averaged_perceptron_tagger') #tagging words with their parts of speech 

nltk.download('punkt')#to handle for abbreviation word
nltk.download('conll2000')#grouping similar words together based on the nature of the word.
nltk.download('brown')

nltk.download('omw-1.4')#Multilingual Wordnet Data from OMW with newer Wordnet versions

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SantoshKumar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SantoshKumar\AppData\Roaming\nltk_data...
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\SantoshKumar\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SantoshKumar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SantoshKumar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\SantoshKumar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\conll2000.zip.
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\SantoshKumar\AppData\Roaming\nltk_data...
[

True

In [13]:
data[['polarity', 'subjectivity']] = data["cleaned_comments"].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
 
for index, row in data["reviews.text"].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
     
    if neg > pos:
        data.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        data.loc[index, 'sentiment'] = "positive"
    else:
        data.loc[index, 'sentiment'] = "neutral"
         
    data.loc[index, 'neg'] = neg
    data.loc[index, 'neu'] = neu
    data.loc[index, 'pos'] = pos
    data.loc[index, 'compound'] = comp

In [14]:
total_pos = len(data.loc[data['sentiment'] == "positive"])
total_neg = len(data.loc[data['sentiment'] == "negative"])
total_neu = len(data.loc[data['sentiment'] == "neutral"])
total_comments = len(data)
print("Total Positive Tweets % : {:.2f}"
      .format((total_pos/total_comments)*100))
print("Total Negative Tweets % : {:.2f}"
      .format((total_neg/total_comments)*100))
print("Total Neutral Tweets % : {:.2f}"
      .format((total_neu/total_comments)*100))

Total Positive Tweets % : 88.56
Total Negative Tweets % : 7.45
Total Neutral Tweets % : 3.98
